<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src/CE_End_To_End_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import os
import numpy as np
from numpy.random import shuffle

In [18]:
spec_dir = '/content/drive/MyDrive/Research/Crowd Emotion v3/Spectrograms/Log/'
data_dir = '/content/drive/MyDrive/Research/Crowd Emotion v3/Dataset/'

In [6]:
NUMBER_OF_CLIPS = {
    'Approval/': 58,
    'Disapproval/': 50,
    'Neutral/': 71
}

In [16]:
for key, value in NUMBER_OF_CLIPS.items():
    clips = np.arange(1, value + 1)
    shuffle(clips)
    for clip in clips:
        files = os.listdir(spec_dir + key)
        

[35  4 32 55 16  7 15 23 43 13 18 27 28 31 12 26 57 48 37 50 34 24 30  8
 42 51 44 14 19 41 45 25 22 52  6  1 39  5 58  2 20 11 36 17 29 40 53 56
 46 47  9 10 21 33  3 54 49 38]
[ 8 44 26 22 37  2 12 46  1 45 48  3 38  9 49 47 18 11 35 34 42  5 39  7
  4 19 43 17 31 28 13 29 33 27 41 15 40 24 32  6 30 50 14 20 10 21 36 16
 25 23]
[33 57 42 66  2 30 13 12 18 19 67 46 63 59 15 58 70 32 11 31  1  4 65 27
  5 60 48 61 25 40 22 55 43 51 16 29 64 45 56 20  7 21 37 35 17 34 53 28
 49 23 44 24 14 39  3 68 36 26 52  6 69 62  9 38 10 54 50 47 71 41  8]


In [20]:
dirs = os.listdir(spec_dir + 'Approval/')

In [21]:
bla = dirs[0]
bla

'pos_c0033_s00000007.jpg'

In [22]:
bla[5: 9]

'0033'